In [11]:
from pathlib import Path
from skimage import io
import numpy as np
import napari

In [18]:
num = 0
janelia = True

im_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/2018-08-01_" + str(num) + "_first10_quantitative.tif"
im = Path(im_path)
im_og = io.imread(im, plugin="tifffile")
print(f"Image shape: {im_og.shape}")

# read coords
if janelia:
    csv_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/voxel_coords.csv"
    coords = np.genfromtxt(csv_path, delimiter=',')
    coords = coords[10*num:10*(num+1)].astype(int)
    soma_coords = [list(coords[0,:])]
    axon_coords = [list(coords[-1,:])]
else:
    csv_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/my_points/points_" + str(num) + ".csv"
    coords = np.genfromtxt(csv_path, delimiter=',')
    coords = coords[1:].astype(int)
    coords = coords[:,1:]
    soma_coords = [list(coords[-1,:])]
    axon_coords = [list(coords[0,:])]

coords_list = list(coords)
coords_list = [list(c) for c in coords_list]
print(f"coords shape: {coords.shape}")

if num == 0:
    soma_coords += [[226, 643, 92], [233, 459, 120], [439, 500, 62]]
elif num == 7:
    soma_coords += [[335, 178, 39], [1, 418, 322]]

print(f"Soma coords: {soma_coords}")

Image shape: (519, 819, 155)
coords shape: (10, 3)
Soma coords: [[365, 239, 116], [226, 643, 92], [233, 459, 120], [439, 500, 62]]


In [13]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(im_og)
viewer.add_points(coords_list, face_color='green', size=8)

viewer.add_points(soma_coords, face_color='orange', size=8)
viewer.add_points(axon_coords, face_color='red', size=8)
viewer.camera.angles = [0, -90, -90]
napari.run()

# Save as zarr

In [15]:
import zarr

In [16]:
zarr_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/" + str(num) + ".zarr"

z = zarr.zeros(im_og.shape, chunks=im_og.shape, dtype=im_og.dtype)
z = im_og
zarr.save(zarr_path, z)

# Make fragments

In [20]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

In [21]:
sg = state_generation(image_path=zarr_path,
        ilastik_program_path="/Applications/ilastik-1.3.3post3-OSX.app/Contents/ilastik-release/run_ilastik.sh",
        ilastik_project_path="/Users/thomasathey/Documents/mimlab/mouselight/octopus_experiment/octopus_exp.ilp",
        chunk_size=im_og.shape,
        soma_coords=soma_coords)

In [22]:
sg.predict(data_bin="/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/misc/")

Constructing probability  image /Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/0_probs.zarr of shape (519, 819, 155)


Computing Ilastik Predictions: 100%|██████████| 1/1 [14:24<00:00, 864.61s/it]
